## Separate the Parameter of Shipment & adding the box

In [6]:
import json
import pandas as pd
import os

# -----------------------------------------
# 1. Load JSON → convert to DataFrame
# -----------------------------------------
with open('instance_107.json') as f:
    data = json.load(f)

rows = []
for shipment_id, items in data.items():
    for item in items:
        item["shipment_id"] = shipment_id
        rows.append(item)

df = pd.DataFrame(rows)

# -----------------------------------------
# 2. Expand rows based on shipped_qty
# -----------------------------------------
expanded_rows = []

for _, row in df.iterrows():
    for i in range(int(row["shipped_qty"])):
        expanded_rows.append({
            "product_code": row["product_code"],
            "WG": row["product_weight"],
            "lp": row["product_length"],
            "wp": row["product_width"],
            "hp": row["product_height"],
            "shipment_id": row["shipment_id"]
        })

expanded_df = pd.DataFrame(expanded_rows)

# -----------------------------------------
# 3. BASE BOX TYPES
# -----------------------------------------
base_box_data = [
    (1, 28,   15,   15,   6300,   32),
    (2, 42,   32,   26,   34944,  94),
    (3, 45.7, 45.7, 40.6, 84793,  156),
    (4, 61,   46,   61,   171166, 469)
]

# -----------------------------------------
# 4. DUPLICATE EACH BOX TYPE 10 TIMES
# -----------------------------------------
duplicates = 10
box_data = []
current_id = 1

for box in base_box_data:
    for j in range(duplicates):
        box_data.append((
            current_id,    # new unique box ID
            box[1], box[2], box[3], box[4], box[5]
        ))
        current_id += 1

# Now box_data contains 40 entries (4 types × 10 duplicates)

# -----------------------------------------
# 5. Function to convert a shipment to AMPL .dat format
# -----------------------------------------
def make_ampl_dat(df_shipment):
    N = len(df_shipment)

    # ---- Set P ----
    set_P = ["set P :="]
    set_P.append(" ".join(str(i) for i in range(1, N+1)))
    set_P.append(";")

    # ---- Product parameters ----
    param_items = ["param: WG lp wp hp :="]
    for i, row in df_shipment.reset_index(drop=True).iterrows():
        idx = i + 1
        param_items.append(
            f"   {idx} {row['WG']} {row['lp']} {row['wp']} {row['hp']}"
        )
    param_items.append(";")

    # ---- Set B with duplicated boxes ----
    set_B = ["set B :="]
    set_B.append(" ".join(str(b[0]) for b in box_data))
    set_B.append(";")

    # ---- Box parameters (duplicated) ----
    param_box = ["param: LB WB HB CB alpha :="]
    for b in box_data:
        param_box.append(
            f"   {b[0]} {b[1]} {b[2]} {b[3]} {b[4]} {b[5]}"
        )
    param_box.append(";")

    return "\n".join(set_P) + "\n\n" + \
           "\n".join(param_items) + "\n\n" + \
           "\n".join(set_B) + "\n\n" + \
           "\n".join(param_box)

# -----------------------------------------
# 6. Create output folder
# -----------------------------------------
output_folder = "shipment_dat_files"
os.makedirs(output_folder, exist_ok=True)

# -----------------------------------------
# 7. Generate and SAVE `.dat` file per shipment
# -----------------------------------------
for sid in expanded_df["shipment_id"].unique():
    df_sh = expanded_df[expanded_df["shipment_id"] == sid]

    ampl_text = make_ampl_dat(df_sh)

    file_path = os.path.join(output_folder, f"shipment_{sid}.dat")
    with open(file_path, "w") as f:
        f.write(ampl_text)

    print(f"Saved: {file_path}")


Saved: shipment_dat_files\shipment_117839.dat
Saved: shipment_dat_files\shipment_118421.dat
Saved: shipment_dat_files\shipment_118473.dat
Saved: shipment_dat_files\shipment_119113.dat
Saved: shipment_dat_files\shipment_169449.dat
Saved: shipment_dat_files\shipment_178000.dat
Saved: shipment_dat_files\shipment_178017.dat
Saved: shipment_dat_files\shipment_181654.dat
Saved: shipment_dat_files\shipment_181959.dat
